In [1]:
import json

VECTILE_CACHE = {}
def get_vectile_gdf(xtile=19293, ytile=24641, z=16,
                    size=256,layer='all',
                    fformat='json', 
                    cache_dir='../data/vectile_cache/'):
    #check if VEC_CACHE object exists in global namespace
    global VECTILE_CACHE
    try:
        VECTILE_CACHE
    except NameError:
        VECTILE_CACHE = {}
    
    # Check if this tile is in python session memory
    # If so, read from the memory, otherwise read from the disk
    if VECTILE_CACHE.get( (z,xtile,ytile) ):
        if VECTILE_CACHE[(z,xtile,ytile)].get('loaded'):
            return VECTILE_CACHE[(z,xtile,ytile)].get('dframe') #geopandas.gdf
        else:
            return gpd.GeoDataFrame.read_file(VECTILE_CACHE[(z,xtile,ytile)].get('fpath'))
    
    # Request a new tile
    tile_url = f'https://tile.nextzen.org/tilezen/vector/v1/{size}/{layer}/{z}/{xtile}/{ytile}.{fformat}?api_key=GpjLSbvrQsa98MgMMuodzw'
    r = requests.get(url=tile_url)
    if not r.ok:
        raise ValueError('reponse not ok: ', r.status_code)
    data = r.json()
    
    # Write to disk
    fdir = (Path(cache_dir) / f'{size}/{layer}/{z}/{xtile}/{ytile}').resolve()
    if not fdir.exists():
        fdir.mkdir(parents=True)
        print(f'{fdir} created')
    fpath = fdir/ f'vectile_{z}_{xtile}_{ytile}.{fformat}'
    print('Saving to: ', fpath)
    with open(fpath, 'w') as f:
        json.dump(data,f)
        
    while not fpath.exists():
        time.sleep(0.3)
    if fpath.exists():
        gdf = gpd.read_file(fpath)
    else:
        raise IOError('File was not correctly written to disk: ', fpath)
    
    # Write to cache
    VECTILE_CACHE[(z,xtile,ytile)] = {
        'loaded': True,
        'dframe': gdf,
        'fpath': fpath
    }
    return gdf

def test_get_vectile_gdf():
    test_vectile = get_vectile_gdf()
#     print(test_vectile.geometry[0])
#     shapes = {i:gv.Shape(geom) for i,geom in enumerate(test_vectile.geometry)}
    vectiles = hv.NdOverlay({i:gv.Shape(geom) for i,geom in enumerate(test_vectile.geometry)})
    display(vectiles.opts(
        opts.Shape(cmap='viridis'))
           )
    
def get_vectile_overlay_at_xyz(xtile, ytile, z):
    """
    Fetches the vector tile (from python cache or from the local disk or from the web service <- search order)
    and returns a NdOverlay of Shape Elements with a numeric index
    
    kwargs:
    - xtile: (default is 19293) 
    - ytile: (default is 24641)  
    - z: (default is 16) 
    - size: (default is 256) 
    - layer: (default is 'all')
    - fformat: (default is 'json')  
    - cache_dir: (default is '../data/vectile_cache/')
    """
    kwargs.update({"xtile": xtile,
                   "ytile": ytile, 
                   "z": z})
    
    gdf = get_vectile_gdf(**kwargs)
    return hv.NdOverlay({i:gv.Shape(geom) for i,geom in enumerate(gdf.geometry)})

def get_vectile_overlay_at_latlon(lat, lon, z):
    xtile, ytile = VectorTile.deg2tile_xy(lat, lon,z)
    return get_vectile_verlay_at_xyz(xtile, ytile, z)  